# 1. Import Dependencies

In [1]:
#Install SWIG https://sourceforge.net/projects/swig/files/swigwin/swigwin-4.0.2/swigwin-4.0.2.zip/download?use_mirror=ixpeering

In [2]:
# !pip install gym[box2d] pyglet==1.3.2

In [3]:
import gym 
from stable_baselines3 import PPO, TD3
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnNoModelImprovement
import os

# 2. Test Environment

In [4]:
environment_name = "CarRacing-v0"

In [5]:
env = gym.make(environment_name)

In [6]:
episodes = 2
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Track generation: 1073..1349 -> 276-tiles track


c:\Users\Patryk\Documents\earin-labs\venv\lib\site-packages\pyglet\image\codecs\wic.py:289: UserWarning: [WinError -2147417850] Po ustawieniu trybu wątku nie można go zmienić
  warnings.warn(str(err))


Episode:1 Score:-27.272727272727625
Track generation: 1140..1429 -> 289-tiles track
Episode:2 Score:-30.55555555555602


In [7]:
env.close()

In [8]:
env.action_space.sample()

array([0.45766917, 0.7324456 , 0.5414255 ], dtype=float32)

In [9]:
env.observation_space.sample()

array([[[ 96, 209, 133],
        [178, 138, 254],
        [ 74,  45, 163],
        ...,
        [208, 218,  72],
        [123,  93, 108],
        [161, 108, 144]],

       [[ 24,  74,  41],
        [234, 251, 129],
        [199, 168,  33],
        ...,
        [145,  63,  54],
        [246, 130, 247],
        [189,  64,  12]],

       [[ 51, 194, 196],
        [ 66, 117, 115],
        [237,   2,  98],
        ...,
        [ 56, 145, 212],
        [ 52,  12, 194],
        [ 21, 148, 179]],

       ...,

       [[173, 252,  99],
        [215, 114,  55],
        [187, 250,  52],
        ...,
        [177, 116, 224],
        [ 25, 156, 118],
        [246,  35,  11]],

       [[ 89, 107, 216],
        [233, 246, 185],
        [ 19,  48, 249],
        ...,
        [245, 145, 218],
        [ 67, 118,  18],
        [105, 255,  67]],

       [[189, 226, 131],
        [ 40,  78, 165],
        [124, 191, 172],
        ...,
        [ 38,  96, 131],
        [103,  10, 154],
        [228, 172,  92]]

# 3. Train Model

### 3. Adding a callback to the training stage

In [10]:
save_path = os.path.join('Training', 'Saved Models')

In [11]:
stop_callback = StopTrainingOnNoModelImprovement(
    max_no_improvement_evals=10, min_evals=20, verbose=True)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)


In [12]:
log_path = os.path.join('Training', 'Logs')

In [26]:
env2 = DummyVecEnv([lambda: env])
model = TD3("CnnPolicy", env2, verbose=1, tensorboard_log=log_path)

Using cpu device
Wrapping the env in a VecTransposeImage.


c:\Users\Patryk\Documents\earin-labs\venv\lib\site-packages\stable_baselines3\common\buffers.py:220: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 55.32GB > 24.02GB
  warnings.warn(


In [27]:
model.learn(total_timesteps=500000, callback=eval_callback)

Track generation: 1187..1488 -> 301-tiles track
Logging to Training\Logs\PPO_8


e:\Anaconda\envs\openai2\lib\site-packages\stable_baselines3\common\callbacks.py:345: UserWarning: Training and eval env are not of the same type<stable_baselines3.common.vec_env.vec_transpose.VecTransposeImage object at 0x0000019D77F7CF70> != <stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv object at 0x0000019D77F7CD60>
  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")


Track generation: 1210..1517 -> 307-tiles track
Track generation: 1079..1353 -> 274-tiles track
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 1e+03    |
|    ep_rew_mean     | -58.8    |
| time/              |          |
|    fps             | 144      |
|    iterations      | 1        |
|    time_elapsed    | 14       |
|    total_timesteps | 2048     |
---------------------------------
Track generation: 1128..1414 -> 286-tiles track
Track generation: 1172..1469 -> 297-tiles track
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 1e+03        |
|    ep_rew_mean          | -55.2        |
| time/                   |              |
|    fps                  | 83           |
|    iterations           | 2            |
|    time_elapsed         | 48           |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.00611673

c:\Users\Patryk\Documents\earin-labs\venv\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Track generation: 1143..1439 -> 296-tiles track
Track generation: 1051..1318 -> 267-tiles track
Track generation: 1076..1372 -> 296-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1032..1294 -> 262-tiles track
Track generation: 1215..1523 -> 308-tiles track
Track generation: 1379..1728 -> 349-tiles track
Eval num_timesteps=10000, episode_reward=-96.38 +/- 18.25
Episode length: 392.80 +/- 220.66
----------------------------------------
| eval/                   |            |
|    mean_ep_length       | 393        |
|    mean_reward          | -96.4      |
| time/                   |            |
|    total_timesteps      | 10000      |
| train/                  |            |
|    approx_kl            | 0.01099097 |
|    clip_fraction        | 0.114      |
|    clip_range           | 0.2        |
|    entropy_loss         | -4.15      |
|    explained_variance   | 0.127      |
|    learning_rate        | 0.0003     |
|    lo

# 4. Save Model 

In [28]:
ppo_path = os.path.join('Training', 'Saved Models', 'PPO_CNNPolicy_Driving_model_500k') # saves the last, not the best

In [29]:
model.save(ppo_path)

# 5. Evaluate and Test

In [31]:
print(evaluate_policy(model, env, n_eval_episodes=20, render=True))

Track generation: 1139..1430 -> 291-tiles track
Track generation: 1299..1627 -> 328-tiles track
Track generation: 1085..1363 -> 278-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1131..1418 -> 287-tiles track
Track generation: 1275..1598 -> 323-tiles track
Track generation: 1339..1677 -> 338-tiles track
Track generation: 1095..1383 -> 288-tiles track
Track generation: 1163..1458 -> 295-tiles track
Track generation: 1087..1369 -> 282-tiles track
Track generation: 945..1193 -> 248-tiles track
Track generation: 1167..1471 -> 304-tiles track
Track generation: 1072..1344 -> 272-tiles track
Track generation: 1172..1475 -> 303-tiles track
Track generation: 1235..1548 -> 313-tiles track
Track generation: 1077..1357 -> 280-tiles track
Track generation: 1125..1410 -> 285-tiles track
Track generation: 993..1250 -> 257-tiles track
Track generation: 1157..1459 -> 302-tiles track
Track generation: 1108..1389 -> 281-tiles track
Track gene

In [28]:
env.close()

# 6. Viewing Logs in Tensroboard

In [19]:
# training_log_path = os.path.join(log_path, 'PPO_4')

In [20]:
# !tensorboard --logdir={training_log_path} --port=6001

In [66]:
episode = 0
while True:
    obs = env.reset()
    score = 0 
    for _ in range(1000):
        action, _states = model.predict(obs.copy(), deterministic=True)
        obs, rewards, dones, info = env.step(action)
        score+=rewards
        env.render()
    episode += 1
    print('Episode:{} Score:{}'.format(episode, score))

Track generation: 1106..1388 -> 282-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1216..1524 -> 308-tiles track
Episode:1 Score:512.3778501628532
Track generation: 1076..1353 -> 277-tiles track
Episode:2 Score:842.0289855072351
Track generation: 1138..1427 -> 289-tiles track
Episode:3 Score:899.9999999999803
Track generation: 1226..1537 -> 311-tiles track
Episode:4 Score:290.3225806451483
Track generation: 1041..1305 -> 264-tiles track
Episode:5 Score:850.5703422053047
Track generation: 1178..1476 -> 298-tiles track
Episode:6 Score:889.898989898973
Track generation: 1194..1496 -> 302-tiles track
Episode:7 Score:899.9999999999907
Track generation: 1071..1343 -> 272-tiles track
Episode:8 Score:899.9999999999893
Track generation: 1204..1509 -> 305-tiles track
Episode:9 Score:656.5789473684067
Track generation: 1220..1529 -> 309-tiles track
Episode:10 Score:721.4285714285556
Track generation: 1216..1524 -> 308-tiles track
Epis

KeyboardInterrupt: 

In [22]:
# env.close()

In [29]:
# del model
# model

In [30]:
# ppo_path = os.path.join('Training', 'Saved Models','the_best_ppo_cnn', 'best_model')
# model = PPO.load(ppo_path, env)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
